In [1]:

import numpy as np


# Define the grid size
grid_size = (5, 5)  # A 5x5 grid


grid = np.zeros(grid_size)


obstacles = [(1, 1), (2, 2), (3, 3)]  # List of obstacle positions


for obstacle in obstacles:
    grid[obstacle] = -1  


start = (0, 0)  # Starting point A
end = (4, 4)  # Destination point B


grid[start] = 1  
grid[end] = 2  







import numpy as np
import gym
from gym import spaces
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque

class GridWorldEnv(gym.Env):
    def __init__(self, grid_size, start, end, obstacles):
        super(GridWorldEnv, self).__init__()
        self.grid_size = grid_size
        self.start = start
        self.end = end
        self.obstacles = obstacles
        self.state = start

        
        self.action_space = spaces.Discrete(4)  # 4 actions: up, down, left, right
        self.observation_space = spaces.Box(low=0, high=max(grid_size), shape=(2,), dtype=int)

    def reset(self):
        self.state = self.start
        return np.array(self.state, dtype=int)

    def step(self, action):
        x, y = self.state
        if action == 0 and y < self.grid_size[1] - 1:  # Up
            y += 1
        elif action == 1 and y > 0:  # Down
            y -= 1
        elif action == 2 and x > 0:  # Left
            x -= 1
        elif action == 3 and x < self.grid_size[0] - 1:  # Right
            x += 1

        next_state = (x, y)

        # Calculate reward
        current_distance = abs(x - self.end[0]) + abs(y - self.end[1])
        new_distance = abs(x - self.end[0]) + abs(y - self.end[1])  

        # Calculate reward
        if next_state in self.obstacles:
            reward = -1  
            done = True
        elif next_state == self.end:
            reward = 10 
            done = True
        else:
            if new_distance > current_distance:
                reward = -1 
            else:
                reward = -0.1  
            done = False
    
        self.state = next_state
        return np.array(self.state, dtype=int), reward, done, {}

    def render(self):
        grid = np.zeros(self.grid_size)
        for obstacle in self.obstacles:
            grid[obstacle] = -1
        grid[self.start] = 1
        grid[self.end] = 2
        grid[self.state] = 3
        print(grid)

# Define Actor and Critic networks
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Actor, self).__init__()
        self.layer1 = nn.Linear(state_dim, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, action_dim)
    
    def forward(self, state):
        x = torch.relu(self.layer1(state))
        x = torch.relu(self.layer2(x))
        x = torch.softmax(self.layer3(x), dim=-1)
        return x

class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()
        self.layer1 = nn.Linear(state_dim + action_dim, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, 1)
    
    def forward(self, state, action):
        x = torch.cat([state, action], 1)
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)
        return x

class DDPGAgent:
    def __init__(self, state_dim, action_dim, max_action):
        self.actor = Actor(state_dim, action_dim).to(device)
        self.actor_target = Actor(state_dim, action_dim).to(device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=1e-3)
        
        self.critic = Critic(state_dim, action_dim).to(device)
        self.critic_target = Critic(state_dim, action_dim).to(device)
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=1e-3)
        
        self.max_action = max_action
        self.replay_buffer = deque(maxlen=100000)
        self.batch_size = 64
        self.discount = 0.99
        self.tau = 0.005
        self.epsilon = 0.2  # Exploration rate
        
    def select_action(self, state, explore=True):
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        if explore and np.random.rand() < self.epsilon:
            action = np.random.randint(0, 4)  # Random action
        else:
            action_probs = self.actor(state).cpu().data.numpy().flatten()
            action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
        return action

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return
        
        batch = random.sample(self.replay_buffer, self.batch_size)
        state, next_state, action, reward, done = zip(*batch)
        
        state = torch.FloatTensor(np.array(state)).to(device)
        next_state = torch.FloatTensor(np.array(next_state)).to(device)
        action = torch.LongTensor(action).to(device).unsqueeze(1)
        reward = torch.FloatTensor(reward).to(device).unsqueeze(1)
        done = torch.FloatTensor(done).to(device).unsqueeze(1)
        
        # One-hot encode actions
        action_one_hot = torch.zeros(self.batch_size, self.actor.layer3.out_features).to(device)
        action_one_hot.scatter_(1, action, 1)
        
        # Compute target Q value
        with torch.no_grad():
            next_action_probs = self.actor_target(next_state)
            next_action = next_action_probs.argmax(dim=1, keepdim=True)
            next_action_one_hot = torch.zeros(self.batch_size, self.actor.layer3.out_features).to(device)
            next_action_one_hot.scatter_(1, next_action, 1)
            target_q = self.critic_target(next_state, next_action_one_hot)
            target_q = reward + ((1 - done) * self.discount * target_q)
        
        # Get current Q estimate
        current_q = self.critic(state, action_one_hot.float())
        
        # Compute critic loss
        critic_loss = nn.MSELoss()(current_q, target_q)
        
        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()
        
        # Compute actor loss
        action_probs = self.actor(state)
        action = action_probs.argmax(dim=1, keepdim=True)
        action_one_hot = torch.zeros(self.batch_size, self.actor.layer3.out_features).to(device)
        action_one_hot.scatter_(1, action, 1)
        actor_loss = -self.critic(state, action_one_hot.float()).mean()
        
        # Optimize the actor
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        # Soft update target networks
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)
        
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)
    
    def add_to_replay_buffer(self, state, next_state, action, reward, done):
        self.replay_buffer.append((state, next_state, action, reward, done))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = GridWorldEnv(grid_size, start, end, obstacles)
agent = DDPGAgent(state_dim=2, action_dim=4, max_action=3)

num_episodes = 100
for episode in range(num_episodes):
    state = env.reset()  # Reset environment for each new episode
    done = False
    episode_reward = 0
    
    print(f"Starting Episode {episode + 1}")
    
    while not done:
        # Select and clip action
        action = agent.select_action(state)
        action = int(np.clip(action, 0, 3))  # Ensure action is within valid range and convert to int
        
        # Execute the action in the environment
        next_state, reward, done, _ = env.step(action)
        
        # Store transition in replay buffer
        agent.add_to_replay_buffer(state, next_state, action, reward, done)
        
        # Train the agent
        agent.train()
        
        # Update state and accumulate reward
        state = next_state
        episode_reward += reward
        
        # Print the grid at each step
        env.render()
    
    # Print final episode information
    print(f"Episode {episode + 1} completed. Total Reward: {episode_reward}")
    print("\n")

print("Training completed!")


Starting Episode 1
[[ 3.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  2.]]
[[ 3.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  2.]]
[[ 1.  3.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  2.]]
[[ 3.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  2.]]
[[ 3.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  2.]]
[[ 1.  3.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  2.]]
[[ 1.  3.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  2.]]
[[ 1.  0.  0.  0.  0.]
 [ 0.  3.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  2.]]
Episode 1 completed. Total Reward: -1.7


Starting Ep